<a href="https://colab.research.google.com/github/aaysan/Facial-Recognition/blob/master/training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing Dependencies

Tensorflow is avaliable in Colab. 

Therefore, the only library I need is PyDrive in order to get the data from my Google Drive.

In [0]:
!pip install PyDrive #install pyDrive


Git repository is cloned for the training. This repo contains all the models as well as the training scripts of Facenet.

In [0]:
!git clone https://github.com/davidsandberg/facenet.git #get the facenet repository

Making sure tensorflow import correctly is my last step of initiliazations. I do this because I had problems with it before and I want to make sure everything works correctly until this point.

I have received two errors very common errors. One was when I tried to import Keras and one was when I tried to import tensorflow itself. Both seemed to work after reseting the runtimes and re-configuring everything from the beginning.

In [0]:
import tensorflow
import pydrive

The snippet below will acquire a facenet model. This model was used in [here](https://github.com/davidsandberg/facenet/wiki/Classifier-training-of-inception-resnet-v1) and it turns out to be a useful model.

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import os

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


The code below get authentication to enter to Google Drive, then downloads all the training data locally.

In [0]:

file_list = drive.ListFile(
    {'q': "'1Lzy5bEQns67Jpl0BQ31eKlufibdwRzpV' in parents"}).GetList()

local_download_path = os.path.expanduser('/content/face_data/model_dir')
  
try:
  os.makedirs(local_download_path)
except: pass

for f in file_list:
  fname = os.path.join(local_download_path, f['title'])
  print(fname)
  downloaded = drive.CreateFile({'id': f['id']})
  content = downloaded.GetContentFile(fname)
  

# Acquiring the Data


# From scratch






The Python path should be set to the src of facenet repo in order for this program to work.

In [0]:
# import sys
import os
os.environ["PYTHONPATH"] = "/content/facenet/src"

Acuqire all the data using this snippet. The commented out line is used for printing the file names that are downlaoded. I found that I don't need this, but it might be useful if you are trying to acquire it for the first time.

In [0]:
try:
    os.makedirs('/content/face_data/') #make the directory
except: 
    pass

# get the file list within the id given after the parameter 'q'. This ID can be
# found on the Drive as the last part of the whole drive link.
file_list = drive.ListFile(
    {'q': "'1FgxBnqzUKMTqFidfwf8EHpIWanPEJkZn' in parents"}).GetList() 


# acquire all the target folders, which are the folders that still exists
# in the directed repo.
target_folders = []
for file in file_list:
  if file['labels']['trashed'] != True:
    target_folders.append(file)

# Download all the files from that folder
for file in target_folders:
  file_list = drive.ListFile(
      {'q':"'{}' in parents".format(file['id'])}).GetList()
  local_download_path = os.path.expanduser('/content/face_data/' + file['title'])
  
  try:
    os.makedirs(local_download_path)
  except: pass
  
  for f in file_list:
    fname = os.path.join(local_download_path, f['title'])
#     print(fname,end="")
    downloaded = drive.CreateFile({'id': f['id']})
    content = downloaded.GetContentFile(fname)

# From a zip file

In [0]:
import pydrive

file_list = drive.ListFile(
    {'q': "'1eZoN0BR8J6-VMaDK4U73A4pocX5RdqeY' in parents"}).GetList()

local_download_path = os.path.expanduser('/content/face_data/')
  
try:
  os.makedirs(local_download_path)
except: pass

for f in file_list:
  fname = os.path.join(local_download_path, f['title'])
  print(fname)
  downloaded = drive.CreateFile({'id': f['id']})
  content = downloaded.GetContentFile(fname)
  

/content/face_data/aligned_images.zip


In [0]:
os.chdir("/content/face_data")
!unzip aligned_images.zip
os.chdir("/content")

FileNotFoundError: ignored

In [0]:
os.chdir("/content")
!mkdir /content/face_data/training
!mv /content/face_data/content/face_data/aligned_faces/* /content/face_data/training
# !rm -f -r /content/training

# Aligning the Faces

For the snippet below


```
!for N in {1..4}; do \
python src/align/align_dataset_mtcnn.py \
<unalligned_images_path> \
<alligned_images_path> \
--image_size <size> \
--margin <margin> \ 
--random_order \ # this is done for gpu enabled devices
--gpu_memory_fraction 0.25 \ #use 4 processes
& done
```

This code will use 4 processes in order to complete, so it takes shorter than usual.

In [0]:
# !echo $PYTHONPATH
os.chdir('facenet')
!for N in {1..4}; do \
python src/align/align_dataset_mtcnn.py \
/content/face_data/ \
/content/face_data/aligned_faces \
--image_size 182 \
--margin 44 \
--random_order \
--gpu_memory_fraction 0.25 \
& done


In [0]:
!zip -r /content/aligned_images.zip /content/face_data

In [0]:
os.chdir("/content") # make the cwd the content to send the zip file to drive.
file1 = drive.CreateFile({"parents": [{"id": "1eZoN0BR8J6-VMaDK4U73A4pocX5RdqeY"}]})
file1.SetContentFile("aligned_images.zip")
file1.Upload()
os.chdir("/content/facenet") # go back to the right place for the data training

If you want to train and evaluate your data on LFW, you need to download [this](http://vis-www.cs.umass.edu/lfw/lfw.tgz) and align the faces for this aswell.

In [0]:
file_list = drive.ListFile(
    {'q': "'1pzj4L1e2TLAbEiwKyW8vnAhoYNYFtLBY' in parents"}).GetList()

local_download_path = os.path.expanduser('/content/face_data/lfw')
  
try:
  os.makedirs(local_download_path)
except: pass

for f in file_list:
  fname = os.path.join(local_download_path, f['title'])
  print(fname)
  downloaded = drive.CreateFile({'id': f['id']})
  content = downloaded.GetContentFile(fname)


In [0]:
os.chdir("/content/face_data")
!mkdir -p lfw/raw
!tar xvf /content/face_data/lfw/lfw.tgz -C lfw/raw --strip-components=1

In [0]:
!ls

In [0]:
os.chdir("lfw/raw")
!ls

In [0]:
os.chdir("/content/facenet")
!for N in {1..4}; do \
python src/align/align_dataset_mtcnn.py \
/content/face_data/lfw/raw \
/content/face_data/lfw_aligned \
--image_size 182 \
--margin 44 \
--random_order \
--gpu_memory_fraction 0.25 \
& done

# Training the Data

In Python 3, the dict.itermitems() attribute was removed and dict.items() was replaced with it. Therefore, I changed and upload this to the main directory. I had to move it to the right place before starting the training.

The classifier function had a similar problem where an assert statement was always true, so I had to remove the parentheses.  

**Changes**


1.   train_softmax.py Line 260: **for key, value in stat.iter_items(): --> for key, value in stat.items(): **
2.   classifier.py Line 59: **assert (len(cls.image_paths)>0, 'There must be at least one image for each class in the dataset') --> assert len(cls.image_paths)>0, 'There must be at least one image for each class in the dataset'**

In [0]:
!mv /train_softmax.py /content/facenet/src/train_softmax.py
!mv /classifier.py /content/facenet/src/classifier.py

In [0]:
!mv /learning_rate_schedule_classifier_haxon.txt /content/facenet/data/learning_rate_schedule_classifier_haxon.txt 

The explanations for the snippet below can be found  [here](https://github.com/davidsandberg/facenet/wiki/Classifier-training-of-inception-resnet-v1).

This will create a model, which can be later used for further training classiying the faces that I want to use. The model will be created in log_base_dir specified. For further details, you can visit the link mentioned above.

Pretrained model parameter should only be used if you have had a previous training data checkpoint.

In [44]:
os.chdir("/content/facenet")
!python src/train_softmax.py \
--logs_base_dir /content/face_data/logs/facenet/ \
--models_base_dir /content/face_data/model_dir/ \
--data_dir /content/face_data/training/ \
--image_size 160 \
--pretrained_model /content/face_data/model_dir/20190226-005412/model-20190226-005412.ckpt-3 \
--model_def models.inception_resnet_v1 \
--optimizer ADAM \
--learning_rate -1 \
--max_nrof_epochs 150 \
--keep_probability 0.8 \
--random_crop \
--random_flip \
--use_fixed_image_standardization \
--learning_rate_schedule_file data/learning_rate_schedule_classifier_casia.txt \
--weight_decay 5e-4 \
--embedding_size 512 \
--lfw_use_flipped_images \
--lfw_subtract_mean \
--validation_set_split_ratio 0.2 \
--validate_every_n_epochs 5 \
--prelogits_norm_loss_factor 5e-4

Model directory: /content/face_data/model_dir/20190226-030607
Log directory: /content/face_data/logs/facenet/20190226-030607
Pre-trained model: /content/face_data/model_dir/20190226-005412/model-20190226-005412.ckpt-3
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.range(limit).shuffle(limit).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensors(tensor).repeat(num_epochs)`.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructi

KeyboardInterrupt: ignored

In [0]:
!python /content/facenet/src/classifier.py \
TRAIN \
/content/face_data/final_train_data/ \
/content/face_data/model_dir/ \
/content/face_data/model_dir/my_model.pkl \
--use_split_dataset \
--batch_size 90 \
--image_size 160 \
--nrof_train_images_per_class 100

In [0]:
!python /content/facenet/src/classifier.py \
CLASSIFY \
/content/facenet/tmp_test/ \
/content/face_data/model_dir/ \
/content/face_data/model_dir/my_model.pkl \
--test_data_dir /content/facenet/tmp_test/ \
--image_size 160 \